# Geodata.cutout updates

### Jeffrey Feng, 8/16/2021

In [1]:
import geodata
import logging
import xarray as xr
logging.basicConfig(level=logging.INFO)

### Load dataset, cutout

In [2]:
DS_monthly_chn = geodata.Dataset(module="merra2",
                                 years=slice(2011, 2011),month=slice(1,12),weather_data_config = "slv_radiation_monthly")
if DS_monthly_chn.prepared == False: DS_monthly_chn.get_data()
DS_monthly_chn.trim_variables()
cutout = geodata.Cutout(name = "china-2011-slv-test",module = "merra2",weather_data_config = "slv_radiation_monthly",
                        xs = slice(73, 136),ys = slice(18, 54),years = slice(2011, 2011), months = slice(1,12))
cutout.prepare()

INFO:geodata.dataset:No months specified, defaulting to 1-12
INFO:geodata.dataset:Using global bounds.
INFO:geodata.dataset:Directory D:/Users/davison_lab_data/merra2 found, checking for completeness.
INFO:geodata.dataset:Directory complete.
INFO:geodata.cutout:All cutout (china-2011-slv-test, D:/Users/davison_lab_data/cutouts) files available.
INFO:geodata.cutout:Cutout subset prepared: <Cutout china-2011-slv-test x=73.12-135.62 y=18.00-54.00 time=2011/1-2011/12 prepared>
INFO:geodata.preparation:The cutout is already prepared. If you want to recalculate it, supply an `overwrite=True` argument.


True

**NOTE:** 

If you have created the mask `china_bin`, please ignore the block below, and run the next section. Otherwise, un-comment the cell below and run it.

In [3]:
# prov_path = shpreader.natural_earth(resolution='10m', category='cultural', name = 'admin_1_states_provinces')
# china_all_shapes = geodata.mask.get_shape(prov_path, key = 'name_en', return_dict = True,
#                          condition_key = 'admin', condition_value = 'China')
# # #get rid of the islands, EDGE CASE cuz the islands are outside of shape
# china_all_shapes.pop(None) 

# china_bin = geodata.Mask("china_bin")
# china_bin.add_layer('FINAL_GRID_5BINS.tif', layer_name = 'bins')

# #extracted shape on bins layer
# china_bin.extract_shapes(china_all_shapes, layer = 'bins')
# china_bin.save_mask()

If you have ran the previous cell before, please comment out the cell above and run the cell below.

### Add mask variable

The mask "china_bin" has no merged_mask value, but the `add_mask` will look for both `merged_mask` and `shape_mask` attribute saved for the mask, unless you set the parameter `merged_mask = False`, or `shape_mask = False`

In [4]:
cutout.add_mask("china_bin")

INFO:geodata.mask:Layer ['bins'] loaded to the mask china_bin.
INFO:geodata.mask:No Merged Mask found.
INFO:geodata.mask:Shape mask ['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong', 'Guangxi Zhuang Autonomous Region', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu', 'Jiangxi', 'Jilin', 'Liaoning', 'Ningxia Hui Autonomous Region', 'Qinghai', 'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin', 'Tibet', 'Xinjiang', 'Yunnan', 'Zhejiang'] loaded to the mask china_bin.
INFO:geodata.mask:Please close the shape_mask xarray(s) for further changes of the mask object.
INFO:geodata.cutout:Cutout.shape_mask added.


In [5]:
#cutout.merged_mask, cutout.shape_mask

### Add grid area variable

In [6]:
cutout.add_grid_area()

In [7]:
#cutout.area

### Generate PV data (cutout conversion)

In [8]:
ds_solar = geodata.convert.pv(cutout, panel = "KANENA", orientation = "latitude_optimal")
ds_solar = ds_solar.reset_coords(['lon', 'lat'], drop = True)
ds_solar = ds_solar.rename({'x': 'lon', 'y': 'lat'})
ds_cutout = ds_solar.to_dataset(name = 'solar')
ds_cutout = ds_cutout.coarsen(time = 12, boundary = 'exact').mean()
ds_cutout = ds_cutout.transpose("time", "lat", "lon")

## Combining variables

In [9]:
combine = cutout.apply_mask(dataset = ds_cutout)

INFO:geodata.cutout:shape_mask combined with dataset. 


In [10]:
combine.keys()

dict_keys(['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong', 'Guangxi Zhuang Autonomous Region', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu', 'Jiangxi', 'Jilin', 'Liaoning', 'Ningxia Hui Autonomous Region', 'Qinghai', 'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin', 'Tibet', 'Xinjiang', 'Yunnan', 'Zhejiang'])

In [13]:
combine['Anhui']

<xarray.Dataset>
Dimensions:  (lat: 73, lon: 101, time: 1)
Coordinates:
  * lat      (lat) float64 18.0 18.5 19.0 19.5 20.0 ... 52.0 52.5 53.0 53.5 54.0
  * time     (time) datetime64[ns] 2011-06-16T12:30:00
  * lon      (lon) float64 73.12 73.75 74.38 75.0 ... 133.8 134.4 135.0 135.6
Data variables:
    solar    (time, lat, lon) float64 0.0 0.0 0.0 ... 0.09313 0.09343 0.09398
    mask     (lat, lon) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    area     (lat, lon) float64 3.663e+03 3.663e+03 ... 2.281e+03 2.281e+03